# CVS and Excel files - Structured data

In [3]:
import pandas as pd
import os


In [4]:
# creating sample data
data = {
    "Product": ["Laptop", "Mouse", "Keyboard", "Monitor", "Webcam"],
    "Category": ["Electronics", "Accessories", "Accessories","Electronics","Electronics"],
    "Price": [999.99, 29.99, 79.99, 299.99, 89.99],
    "Stock": [50, 200, 150, 75, 100],
    "Description": [
        "High performance laptop with 16gb ram and 512 gb ssd",
        "Wireless optical mouse with ergonomic design",
        "mechanical keyboard with rgb backlighting",
        "27-inch 4k monitor with hdr support",
        "1080p webcam with noise cancellation"
    ]
}

df = pd.DataFrame(data)
df.to_csv("data/structured_files/products.csv", index=False)

In [6]:
with pd.ExcelWriter("data/structured_files/inventory.xlsx") as writer:
    df.to_excel(writer, sheet_name="Products", index=False)

    summary_data = {
        "Category": ["Electronics", "Accessories"],
        "Total_Items": [3,2],
        "Total_Value": [1389.97, 109.98]
    }

    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)

# CSV Processing

In [7]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

e:\Babu\ragudemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CSVLoader: Row-based documents

In [10]:
csv_loader = CSVLoader(
    file_path="data/structured_files/products.csv",
    encoding="utf-8",
    csv_args={
        "delimiter": ",",
        "quotechar": '"'
    }
)

csv_docs = csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)} documents(one per row)")
print(f"\nFirst document")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High performance laptop with 16gb ram and 512 gb ssd'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: mechanical keyboard with rgb backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4k monitor with hdr support'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 4}, page_content='Product: Webcam\nCategory: Electr

### Method 2: custom csv loader with extra metadata

In [13]:
from typing import List
from langchain_core.documents import Document

In [19]:
def process_csv_intelligently(filepath: str) -> List[Document]:
    df = pd.read_csv(filepath)
    documents = []
    for idx, row in df.iterrows():
        content = f"""Product Information:
Name: {row['Product']}
Category: {row['Category']}
Price: ${row['Price']}
Stock: {row['Stock']} units
Description: {row['Description']}"""
        
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)

    return documents


In [18]:
process_csv_intelligently("data/structured_files/products.csv")

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'data_type': 'product_info'}, page_content='Product Information:\nName: Laptop\nCategory: Electronics\nPrice: $999.99\nStock: 50 units\nDescription: High performance laptop with 16gb ram and 512 gb ssd'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'data_type': 'product_info'}, page_content='Product Information:\nName: Mouse\nCategory: Accessories\nPrice: $29.99\nStock: 200 units\nDescription: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'data_type': 'product_info'}, page_content='Product Information:\nName: Keyboard\nCategory: Accessories\nPrice: $79.99\nStock: 150 units\nDescription: mechanical keyboard with rgb backlig

CSV Processing Strategies

1. Row based (CSVLoader)
    - Simple one-row-one document
    - Good for record lookups
    - Looses table context

2. Intelligent Processing (custom)
    - Preserves relationships
    - Creates summaries
    - Rich metadata
    - Better for Q&A

## Excel processing

### Method 1: Using pandas for full-control

In [20]:
def process_excel_with_pandas(filepath: str) -> List[Document]:
    documents = []

    excel_file = pd.ExcelFile(filepath)

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)

        # create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {len(df)}\n\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)

        docs = Document(
            page_content = sheet_content,
            metadata = {
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(docs)

    return documents

In [21]:
process_excel_with_pandas("data/structured_files/inventory.xlsx")

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumns: 5\n\nRows: 5\n\n Product    Category  Price  Stock                                          Description\n  Laptop Electronics 999.99     50 High performance laptop with 16gb ram and 512 gb ssd\n   Mouse Accessories  29.99    200         Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150            mechanical keyboard with rgb backlighting\n Monitor Electronics 299.99     75                  27-inch 4k monitor with hdr support\n  Webcam Electronics  89.99    100                 1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: 2\n\nRows: 2\n\n   Category  Total_Items  Total_Value\

## Method 2: UnstructuredExcelLoader

In [22]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [25]:
try:
    excel_loader = UnstructuredExcelLoader(
        "data/structured_files/inventory.xlsx",
        mode="elements"
    )
    unstrutured_docs = excel_loader.load()
    print("Handles complex features")
    print("Preserves formatting info")
    print("Required unstructured library")
except Exception as e:
    raise Exception(str(e))

Handles complex features
Preserves formatting info
Required unstructured library


In [26]:
unstrutured_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'file_directory': 'data/structured_files', 'filename': 'inventory.xlsx', 'last_modified': '2025-11-15T21:24:35', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High performance laptop with 16gb ram and 512 gb ssd</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.99</td><td>200</td><td>Wireless optical mouse with ergonomic design</td></tr><tr><td>Keyboard</td><td>Accessories</td><td>79.99</td><td>150</td><td>mechanical keyboard with rgb backlighting</td></tr><tr><td>Monitor</td><td>Electronics</td><td>299.99</td><td>75</td><td>27-inch 4k monitor with hdr support</td></tr><tr><td>Webcam</td><td>Electronics</td><td>89.99</td><td>100</td><td>1080p webcam with noise cancellation</td></tr></table>', 'languages': ['eng'], 'filetype': 'applicati